# Forecasting air pollution with LSTM
### result    
- **314 raw air pollution dataset for 24 hours**, 1000 epoch with 256 batch size  
Train RMSE : 13.068013921498887  
Test RMSE : 28.917407644497754  
RMSE of 314 dataset : 17.372590229479655  
  

- **10000 interpolated air pollution dataset**  
Train RMSE : 2.3552439778752547  
Test RMSE : 3.8665034729314485  
RMSE of 314 dataset : 38.37646593777526  

## 0. Import Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,  Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

## 1. Make Dataset

In [72]:
dataset = np.zeros((314,1))

for hour in range(24) :
    path = os.path.join(os.getcwd(), 'air_pollution_data', 'Hour_{}'.format(hour+1) + '.csv')
    csv = pd.read_csv(path, encoding = 'cp949', header = None)
    csv.head()
    data = csv.values[:,-1]
    data = data.reshape(314,1)
    
    dataset = np.concatenate((dataset, data), axis = 1)
    
dataset = dataset[:, 1:]    

In [73]:
dataset = dataset.reshape(314,1,24)
print("Shape of dataset :", dataset.shape)

Shape of dataset : (314, 1, 24)


In [74]:
x = dataset[:, :, :23]
y = dataset[:, :, -1]

In [75]:
print("Shape of x :", x.shape)
print("Shape of y :", y.shape)

Shape of x : (314, 1, 23)
Shape of y : (314, 1)


In [76]:
x_train = x[:251,:,:]
y_train = y[:251,:]

x_test = x[252:,:,:]
y_test = y[252:,:]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(251, 1, 23) (251, 1)
(62, 1, 23) (62, 1)


## 2. LSTM

In [101]:
model = Sequential()
model.add(LSTM(50, input_shape = (1, 23), return_sequences=False))
# model.add(LSTM(128,  return_sequences=False))

# model.add(Dense(128, activation = "relu"))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation = "relu"))
# model.add(Dropout(0.5))
# model.add(Dense(32, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation = "linear"))

model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_22 (LSTM)               (None, 50)                14800     
_________________________________________________________________
dropout_30 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 51        
Total params: 14,851
Trainable params: 14,851
Non-trainable params: 0
_________________________________________________________________


In [102]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [103]:
losses = tf.compat.v1.losses.log_loss
model.compile(loss = root_mean_squared_error, optimizer='adam')

In [104]:
LSTM_model = model.fit(x_train, y_train,
                       epochs = 100,
                       batch_size = 1,
                       verbose = 1, 
                       validation_split=0.2)

Train on 200 samples, validate on 51 samples
Epoch 1/100
200/200 [==============================] - 3s 13ms/sample - loss: 67.9968 - val_loss: 73.7277
Epoch 2/100
200/200 [==============================] - 1s 3ms/sample - loss: 62.3675 - val_loss: 69.5590
Epoch 3/100
200/200 [==============================] - 1s 3ms/sample - loss: 59.1430 - val_loss: 66.3326
Epoch 4/100
200/200 [==============================] - 1s 3ms/sample - loss: 56.5287 - val_loss: 63.7976
Epoch 5/100
200/200 [==============================] - 1s 3ms/sample - loss: 54.2428 - val_loss: 60.4466
Epoch 6/100
200/200 [==============================] - 1s 3ms/sample - loss: 51.6334 - val_loss: 57.2818
Epoch 7/100
200/200 [==============================] - 1s 3ms/sample - loss: 49.2035 - val_loss: 54.2141
Epoch 8/100
200/200 [==============================] - 1s 3ms/sample - loss: 46.5960 - val_loss: 51.3214
Epoch 9/100
200/200 [==============================] - 1s 3ms/sample - loss: 44.1702 - val_loss: 49.3011
Epoch 10/

Epoch 78/100
200/200 [==============================] - 1s 3ms/sample - loss: 17.5774 - val_loss: 24.1051
Epoch 79/100
200/200 [==============================] - 1s 3ms/sample - loss: 18.6579 - val_loss: 25.4584
Epoch 80/100
200/200 [==============================] - 1s 3ms/sample - loss: 17.5306 - val_loss: 24.9057
Epoch 81/100
200/200 [==============================] - 1s 3ms/sample - loss: 19.4585 - val_loss: 24.7385
Epoch 82/100
200/200 [==============================] - 1s 3ms/sample - loss: 18.2098 - val_loss: 24.5966
Epoch 83/100
200/200 [==============================] - 1s 3ms/sample - loss: 18.8625 - val_loss: 24.6669
Epoch 84/100
200/200 [==============================] - 1s 3ms/sample - loss: 18.9837 - val_loss: 24.7128
Epoch 85/100
200/200 [==============================] - 1s 3ms/sample - loss: 17.7692 - val_loss: 24.3794
Epoch 86/100
200/200 [==============================] - 1s 3ms/sample - loss: 18.7631 - val_loss: 25.5244
Epoch 87/100
200/200 [========================

## 3. Result

In [100]:
train_pred = model.predict(x_train)
test_pred = model.predict(x_test)

train_RMSE = mean_squared_error(y_train, train_pred) ** 0.5
test_RMSE = mean_squared_error(y_test, test_pred) ** 0.5

print('Train RMSE :', train_RMSE)
print('Test RMSE :', test_RMSE)

Train RMSE : 58.4693726064508
Test RMSE : 87.65483153453327


In [94]:
yy = model.predict(x)
result = mean_squared_error(y, yy) ** 0.5
print("RMSE of 314 dataset :", result)

RMSE of 314 dataset : 25.825870447596127


## 4. Train with interpolated dataset

In [95]:
dataset = np.zeros((10000,1))

for hour in range(24) :
    path = os.path.join(os.getcwd(), 'Interpolation_air_pollution', 'Hour_{}'.format(hour+1) + '_inter_p' + '.txt')
    txt = np.genfromtxt(path, delimiter=' ')

    data = np.reshape(txt, (1, -1)).T
    
    dataset = np.concatenate((dataset, data), axis = 1)
    
dataset = dataset[:, 1:] 

In [120]:
dataset = dataset.reshape(10000,1,24)
print("Shape of dataset :", dataset.shape)

Shape of dataset : (10000, 1, 24)


In [121]:
x = dataset[:, :, :23]
y = dataset[:, :, -1]

print("Shape of x :", x.shape)
print("Shape of y :", y.shape)

Shape of x : (10000, 1, 23)
Shape of y : (10000, 1)


In [127]:
x_train = x[:8000,:,:]
y_train = y[:8000,:]

x_test = x[8000:,:,:]
y_test = y[8000:,:]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(8000, 1, 23) (8000, 1)
(2000, 1, 23) (2000, 1)


In [128]:
model2 = Sequential()
model2.add(LSTM(128, input_shape = (1, 23), return_sequences=True))
model2.add(LSTM(128,  return_sequences=False))

model2.add(Dense(128, activation = "relu"))
model2.add(Dropout(0.5))
model2.add(Dense(64, activation = "relu"))
model2.add(Dropout(0.5))
model2.add(Dense(32, activation = "relu"))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation = "linear"))

model2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 1, 128)            77824     
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_16 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)               

In [129]:
losses = tf.compat.v1.losses.log_loss
model2.compile(loss = root_mean_squared_error, optimizer='adam')

In [130]:
LSTM_model2 = model2.fit(x_train, y_train,
                       epochs = 1000,
                       batch_size = 256,
                       verbose = 1, 
                       validation_split=0.2)

Epoch 1/1000
25/25 [==============================] - 1s 36ms/step - loss: 55.3159 - val_loss: 45.4598
Epoch 2/1000
25/25 [==============================] - 0s 7ms/step - loss: 27.2821 - val_loss: 17.2400
Epoch 3/1000
25/25 [==============================] - 0s 7ms/step - loss: 21.3795 - val_loss: 9.7260
Epoch 4/1000
25/25 [==============================] - 0s 8ms/step - loss: 19.5267 - val_loss: 8.4769
Epoch 5/1000
25/25 [==============================] - 0s 9ms/step - loss: 18.7165 - val_loss: 8.0161
Epoch 6/1000
25/25 [==============================] - 0s 8ms/step - loss: 17.7353 - val_loss: 6.9600
Epoch 7/1000
25/25 [==============================] - 0s 8ms/step - loss: 17.1432 - val_loss: 8.7011
Epoch 8/1000
25/25 [==============================] - 0s 8ms/step - loss: 17.5122 - val_loss: 7.5763
Epoch 9/1000
25/25 [==============================] - 0s 8ms/step - loss: 17.1703 - val_loss: 7.0544
Epoch 10/1000
25/25 [==============================] - 0s 7ms/step - loss: 17.2421 - val

Epoch 161/1000
25/25 [==============================] - 0s 7ms/step - loss: 12.1692 - val_loss: 4.8638
Epoch 162/1000
25/25 [==============================] - 0s 7ms/step - loss: 12.2054 - val_loss: 4.6664
Epoch 163/1000
25/25 [==============================] - 0s 7ms/step - loss: 12.3156 - val_loss: 4.3757
Epoch 164/1000
25/25 [==============================] - 0s 7ms/step - loss: 12.3575 - val_loss: 4.3402
Epoch 165/1000
25/25 [==============================] - 0s 7ms/step - loss: 12.2182 - val_loss: 4.7069
Epoch 166/1000
25/25 [==============================] - 0s 7ms/step - loss: 12.3728 - val_loss: 4.0296
Epoch 167/1000
25/25 [==============================] - 0s 7ms/step - loss: 11.9047 - val_loss: 4.3348
Epoch 168/1000
25/25 [==============================] - 0s 7ms/step - loss: 12.0570 - val_loss: 4.4927
Epoch 169/1000
25/25 [==============================] - 0s 7ms/step - loss: 12.2736 - val_loss: 4.4985
Epoch 170/1000
25/25 [==============================] - 0s 8ms/step - los

25/25 [==============================] - 0s 7ms/step - loss: 10.8309 - val_loss: 3.8972
Epoch 320/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.9701 - val_loss: 3.9716
Epoch 321/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.8413 - val_loss: 3.8602
Epoch 322/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.7829 - val_loss: 4.0849
Epoch 323/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.8311 - val_loss: 4.1001
Epoch 324/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.7342 - val_loss: 3.9984
Epoch 325/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.8859 - val_loss: 3.9842
Epoch 326/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.8023 - val_loss: 3.6893
Epoch 327/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.8695 - val_loss: 4.0789
Epoch 328/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.6588 - va

25/25 [==============================] - 0s 7ms/step - loss: 10.1188 - val_loss: 3.9300
Epoch 478/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.1358 - val_loss: 3.6479
Epoch 479/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.1561 - val_loss: 3.9977
Epoch 480/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.1309 - val_loss: 3.9025
Epoch 481/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.2497 - val_loss: 3.6021
Epoch 482/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.1605 - val_loss: 3.8977
Epoch 483/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.0168 - val_loss: 3.9130
Epoch 484/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.2988 - val_loss: 3.7127
Epoch 485/1000
25/25 [==============================] - 0s 7ms/step - loss: 9.9579 - val_loss: 3.8097
Epoch 486/1000
25/25 [==============================] - 0s 7ms/step - loss: 10.0083 - val

25/25 [==============================] - 0s 7ms/step - loss: 9.6684 - val_loss: 3.6808
Epoch 558/1000
25/25 [==============================] - 0s 7ms/step - loss: 9.6697 - val_loss: 3.7558
Epoch 559/1000
25/25 [==============================] - 0s 7ms/step - loss: 9.4899 - val_loss: 3.9074
Epoch 560/1000
25/25 [==============================] - 0s 7ms/step - loss: 9.6105 - val_loss: 3.6096
Epoch 561/1000
25/25 [==============================] - 0s 7ms/step - loss: 9.7846 - val_loss: 3.8139
Epoch 562/1000
25/25 [==============================] - 0s 7ms/step - loss: 9.6473 - val_loss: 3.8216
Epoch 563/1000
25/25 [==============================] - 0s 7ms/step - loss: 9.5199 - val_loss: 3.5140
Epoch 564/1000
25/25 [==============================] - 0s 7ms/step - loss: 9.6148 - val_loss: 3.7630
Epoch 565/1000
25/25 [==============================] - 0s 7ms/step - loss: 9.6299 - val_loss: 3.6885
Epoch 566/1000
25/25 [==============================] - 0s 7ms/step - loss: 9.6983 - val_loss: 3.

25/25 [==============================] - 0s 7ms/step - loss: 8.9889 - val_loss: 3.6036
Epoch 718/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.9253 - val_loss: 3.4764
Epoch 719/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.8908 - val_loss: 3.5363
Epoch 720/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.8461 - val_loss: 3.4975
Epoch 721/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.8437 - val_loss: 3.6506
Epoch 722/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.7036 - val_loss: 3.7597
Epoch 723/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.7870 - val_loss: 3.4634
Epoch 724/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.8887 - val_loss: 3.5841
Epoch 725/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.7521 - val_loss: 3.6510
Epoch 726/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.9067 - val_loss: 3.

25/25 [==============================] - 0s 7ms/step - loss: 8.1384 - val_loss: 3.6386
Epoch 878/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.0397 - val_loss: 3.4729
Epoch 879/1000
25/25 [==============================] - 0s 6ms/step - loss: 8.1932 - val_loss: 3.4758
Epoch 880/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.0977 - val_loss: 3.3599
Epoch 881/1000
25/25 [==============================] - 0s 7ms/step - loss: 7.9144 - val_loss: 3.3559
Epoch 882/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.1499 - val_loss: 3.4540
Epoch 883/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.0982 - val_loss: 3.6069
Epoch 884/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.0886 - val_loss: 3.3244
Epoch 885/1000
25/25 [==============================] - 0s 7ms/step - loss: 8.1012 - val_loss: 3.3686
Epoch 886/1000
25/25 [==============================] - 0s 6ms/step - loss: 7.9687 - val_loss: 3.

In [136]:
train_pred = model2.predict(x_train)
test_pred = model2.predict(x_test)

train_RMSE = mean_squared_error(y_train, train_pred) ** 0.5
test_RMSE = mean_squared_error(y_test, test_pred) ** 0.5

print('Train RMSE :', train_RMSE)
print('Test RMSE :', test_RMSE)

Train RMSE : 2.3552439778752547
Test RMSE : 3.8665034729314485


In [137]:
yy = model2.predict(x)
result = mean_squared_error(y, yy) ** 0.5
print("RMSE of 314 dataset :", result)

RMSE of 314 dataset : 38.37646593777526
